In [1]:
import plotly.graph_objects as go
import numpy as np

In [2]:
with open('Meshes/dumbell.obj', 'r') as reader:
    obj_data = reader.read()

In [3]:
def obj_data_to_mesh3d(odata):
    # odata is the string read from an obj file
    vertices = []
    faces = []
    lines = odata.splitlines()   
   
    for line in lines:
        slist = line.split()
        if slist:
            if slist[0] == 'v':
                vertex = np.array(slist[1:], dtype=float)
                vertices.append(vertex)
            elif slist[0] == 'f':
                face = []
                for k in range(1, len(slist)):
                    face.append([int(s) for s in slist[k].replace('//','/').split('/')])
                if len(face) > 3: # triangulate the n-polyonal face, n>3
                    faces.extend([[face[0][0]-1, face[k][0]-1, face[k+1][0]-1] for k in range(1, len(face)-1)])
                else:    
                    faces.append([face[j][0]-1 for j in range(len(face))])
            else: pass
    
    
    return np.array(vertices), np.array(faces)   

In [4]:
vertices, faces = obj_data_to_mesh3d(obj_data)

In [5]:
vertices

array([[-2.      ,  0.      ,  2.      ],
       [-2.      ,  4.      ,  2.      ],
       [-2.      ,  0.      ,  0.      ],
       [-2.      ,  4.      ,  0.      ],
       [ 0.      ,  0.      ,  2.      ],
       [ 0.      ,  4.      ,  2.      ],
       [ 0.      ,  0.      ,  0.      ],
       [ 0.      ,  4.      ,  0.      ],
       [-1.549541,  2.      ,  1.549541],
       [-1.549541,  2.      ,  0.450459],
       [-0.450459,  2.      ,  0.450459],
       [-0.450459,  2.      ,  1.549541]])

In [6]:
faces

array([[ 8,  1,  3],
       [ 8,  3,  9],
       [ 9,  3,  7],
       [ 9,  7, 10],
       [10,  7,  5],
       [10,  5, 11],
       [11,  5,  1],
       [11,  1,  8],
       [ 2,  6,  4],
       [ 2,  4,  0],
       [ 7,  3,  1],
       [ 7,  1,  5],
       [ 4, 11,  8],
       [ 4,  8,  0],
       [ 6, 10, 11],
       [ 6, 11,  4],
       [ 2,  9, 10],
       [ 2, 10,  6],
       [ 0,  8,  9],
       [ 0,  9,  2]])

In [7]:
def edges_from_faces(faces):
    # eine liste von kanten, definiert durch indized von zwei benachbarten faces
    # bsp:[[1,2],[5,6]] face[1] und face[2] sind benachbart 
    # und sollten jeweils zwei vertexe haben, die sie gemeinsamm haben
    edges_byface = []
    edges_byvertex = []
    for i,face1 in enumerate(faces):
        if(i==faces.size-1):
            break
        neighbor_count = 0
        for j, face2 in enumerate(faces[i+1:,]):
            intersect = np.intersect1d(face1,face2)
            if (intersect.size == 2):
                # edges.append(intersect) fuer eine list von kanten dargestellt mit vertexe
                # unten wird eine Kante mit zwei faces dargestellt
                edges_byface.append([i,i+j+1])
                edges_byvertex.append(intersect.tolist())
                neighbor_count += 1
            if(neighbor_count == 3):
                break
    return np.array(edges_byface), np.array(edges_byvertex)         

In [8]:
edges_byface, edges_byvertex= edges_from_faces(faces)
edges_byface.shape

(30, 2)

In [9]:
edges_byface

array([[ 0,  1],
       [ 0,  7],
       [ 0, 10],
       [ 1,  2],
       [ 1, 18],
       [ 2,  3],
       [ 2, 10],
       [ 3,  4],
       [ 3, 16],
       [ 4,  5],
       [ 4, 11],
       [ 5,  6],
       [ 5, 14],
       [ 6,  7],
       [ 6, 11],
       [ 7, 12],
       [ 8,  9],
       [ 8, 15],
       [ 8, 17],
       [ 9, 13],
       [ 9, 19],
       [10, 11],
       [12, 13],
       [12, 15],
       [13, 18],
       [14, 15],
       [14, 17],
       [16, 17],
       [16, 19],
       [18, 19]])

In [11]:
edges_byvertex

array([[ 3,  8],
       [ 1,  8],
       [ 1,  3],
       [ 3,  9],
       [ 8,  9],
       [ 7,  9],
       [ 3,  7],
       [ 7, 10],
       [ 9, 10],
       [ 5, 10],
       [ 5,  7],
       [ 5, 11],
       [10, 11],
       [ 1, 11],
       [ 1,  5],
       [ 8, 11],
       [ 2,  4],
       [ 4,  6],
       [ 2,  6],
       [ 0,  4],
       [ 0,  2],
       [ 1,  7],
       [ 4,  8],
       [ 4, 11],
       [ 0,  8],
       [ 6, 11],
       [ 6, 10],
       [ 2, 10],
       [ 2,  9],
       [ 0,  9]])

In [17]:
edges = vertices[edges_byvertex]
edges

array([[[-2.      ,  4.      ,  0.      ],
        [-1.549541,  2.      ,  1.549541]],

       [[-2.      ,  4.      ,  2.      ],
        [-1.549541,  2.      ,  1.549541]],

       [[-2.      ,  4.      ,  2.      ],
        [-2.      ,  4.      ,  0.      ]],

       [[-2.      ,  4.      ,  0.      ],
        [-1.549541,  2.      ,  0.450459]],

       [[-1.549541,  2.      ,  1.549541],
        [-1.549541,  2.      ,  0.450459]],

       [[ 0.      ,  4.      ,  0.      ],
        [-1.549541,  2.      ,  0.450459]],

       [[-2.      ,  4.      ,  0.      ],
        [ 0.      ,  4.      ,  0.      ]],

       [[ 0.      ,  4.      ,  0.      ],
        [-0.450459,  2.      ,  0.450459]],

       [[-1.549541,  2.      ,  0.450459],
        [-0.450459,  2.      ,  0.450459]],

       [[ 0.      ,  4.      ,  2.      ],
        [-0.450459,  2.      ,  0.450459]],

       [[ 0.      ,  4.      ,  2.      ],
        [ 0.      ,  4.      ,  0.      ]],

       [[ 0.      ,  4.      ,  

In [15]:
tris = vertices[faces]
tris

array([[[-1.549541,  2.      ,  1.549541],
        [-2.      ,  4.      ,  2.      ],
        [-2.      ,  4.      ,  0.      ]],

       [[-1.549541,  2.      ,  1.549541],
        [-2.      ,  4.      ,  0.      ],
        [-1.549541,  2.      ,  0.450459]],

       [[-1.549541,  2.      ,  0.450459],
        [-2.      ,  4.      ,  0.      ],
        [ 0.      ,  4.      ,  0.      ]],

       [[-1.549541,  2.      ,  0.450459],
        [ 0.      ,  4.      ,  0.      ],
        [-0.450459,  2.      ,  0.450459]],

       [[-0.450459,  2.      ,  0.450459],
        [ 0.      ,  4.      ,  0.      ],
        [ 0.      ,  4.      ,  2.      ]],

       [[-0.450459,  2.      ,  0.450459],
        [ 0.      ,  4.      ,  2.      ],
        [-0.450459,  2.      ,  1.549541]],

       [[-0.450459,  2.      ,  1.549541],
        [ 0.      ,  4.      ,  2.      ],
        [-2.      ,  4.      ,  2.      ]],

       [[-0.450459,  2.      ,  1.549541],
        [-2.      ,  4.      ,  2.      

In [ ]:
def angular_distance(edges,):
    
    

In [ ]:
n = np.cross( tris[::,1 ] - tris[::,0]  , tris[::,2 ] - tris[::,0] )
n

In [ ]:
def normalize_v3(arr):
    lens = np.sqrt( arr[:,0]**2 + arr[:,1]**2 + arr[:,2]**2 )
    arr[:,0] /= lens
    arr[:,1] /= lens
    arr[:,2] /= lens                
    return arr

In [ ]:
normalize_v3(n)